## Training an MNIST (Modified National Institute of Standards and Technology database) Model

#### Training a deep learning model, CNN, to recoginize handwritten digits

In [2]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

In [12]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.one_hot(y_train, 10).numpy()
y_test = tf.one_hot(y_test, 10).numpy()

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
train_dataset = dc.data.NumpyDataset(x_train, y_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test)

In [5]:
features = tf.keras.Input(shape=(28,28,1))

In [6]:
conv2d_1 = layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu)(features)
conv2d_2 = layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu)(conv2d_1)

In [7]:
flatten = layers.Flatten()(conv2d_2)
dense1 = layers.Dense(units=1024, activation=tf.nn.relu)(flatten)
dense2 = layers.Dense(units=10, activation=None)(dense1)

In [8]:
output = layers.Activation(tf.math.softmax)(dense2)

In [9]:
keras_model = tf.keras.Model(inputs=features, outputs=[output, dense2])

In [10]:
model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SoftmaxCrossEntropy(),
    output_types=['prediction', 'loss'],
    model_dir='mnist')

In [11]:
model.fit(train_dataset, nb_epoch=10)

0.010838865041732789

In [15]:
metric = dc.metrics.Metric(dc.metrics.accuracy_score)

In [16]:
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])

In [17]:
print(train_scores)
print(test_scores)

{'accuracy_score': 0.9994016666666667}
{'accuracy_score': 0.99648}


In [18]:
# Done